# TWEETS

In [15]:
import sys
import codecs
import math #for computemean
from os import listdir
from os.path import isfile, join
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
from copy import deepcopy
from string import punctuation
from random import shuffle

import gensim
from gensim.models.word2vec import Word2Vec # the word2vec model gensim class
LabeledSentence = gensim.models.doc2vec.LabeledSentence

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
tokenizer = TweetTokenizer()


#sklearn
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import scale

#zemberek turkish library, change with your path
import jpype
from jpype import *
if not jpype.isJVMStarted():
    jpype.startJVM(getDefaultJVMPath(),"-Djava.class.path=/Users/asus/Desktop/Sentiment Analysis/zemberek-tum-2.0.jar", "-ea")
Tr = jpype.JClass("net.zemberek.tr.yapi.TurkiyeTurkcesi")
tr = Tr()
Zemberek = jpype.JClass("net.zemberek.erisim.Zemberek")
zemberek = Zemberek(tr)

In [16]:
path1 = "3000tweet/raw_texts/1"
path2 = "3000tweet/raw_texts/2"
path3 = "3000tweet/raw_texts/3"


files1 = [f for f in listdir(path1) if isfile(join(path1, f))]
files2 = [f for f in listdir(path2) if isfile(join(path2, f))]
files3 = [f for f in listdir(path3) if isfile(join(path3, f))]
totalFileCount = len(files1) + len(files2) + len(files3)
print("Positive tweet count:",len(files1))
print("Negative tweet count:",len(files2))
print("Neutral tweet count:",len(files3))
print("Total tweet count:",len(files1) + len(files2) + len(files3))

Positive tweet count: 756
Negative tweet count: 1287
Neutral tweet count: 957
Total tweet count: 3000


In [17]:
df = pd.DataFrame(columns=['Sentiment','SentimentText'],index=range(totalFileCount))

In [18]:

for idx, file in enumerate(files1):
    file = open( path1 + "/" + file, "r", encoding="cp1254") 
    sentimentDict = {}
    sentimentDict["Sentiment"] = 1
    sentimentDict["SentimentText"] =  file.read() 
    df.iloc[idx] = sentimentDict
    
for idx, file in enumerate(files2):
    file = open( path2 + "/" + file, "r", encoding="cp1254") 
    sentimentDict = {}
    sentimentDict["Sentiment"] = 2
    sentimentDict["SentimentText"] =  file.read() 
    df.iloc[len(files1) + idx] = sentimentDict
    
for idx, file in enumerate(files3):
    file = open( path3 + "/" + file, "r", encoding="cp1254") 
    sentimentDict = {}
    sentimentDict["Sentiment"] = 3
    sentimentDict["SentimentText"] =  file.read() 
    df.iloc[len(files1) + len(files2) + idx] = sentimentDict

df.head(10)

,Sentiment,SentimentText
0,1,dun Turkcelle tepkilerimizden sonra bugün Turk...
1,1,girmezmiyim.. Turkcell kartim bile var.. Yarin...
2,1,tam tünelden gecerken 3g cekiyordu:D türkcell'...
3,1,turkcell superonline fiber internet veya ADSL ...
4,1,bence Gnçtrkcll Ark Winterfest 2012'de 1.olur ...
5,1,turkcell sana kıyak geçiyor :D
6,1,Turkcelle bağlan hayata diyorum ;)))
7,1,"Kimse takmıyo beni yaaaaa, turkcell bana mesaj..."
8,1,Bu Turkcell pusula uygulaması iPhone kullanıcı...
9,1,o zaman sorun yok bende turkcell için iyi bir ...


In [19]:
df.shape

(3000, 2)

In [20]:
def tokenize(tweet):
    try:
        tweet = tweet.lower() 
        tokens = tokenizer.tokenize(tweet)
        tokens = [x for x in tokens if not x.startswith('@')]
        tokens = [x for x in tokens if not x.startswith('#')]
        tokens = [x for x in tokens if not x.startswith('http')]
        tokens = [x for x in tokens if not x.startswith('.')]
        tokens = [x for x in tokens if not x.startswith('(')]
        tokens = [x for x in tokens if not x.startswith(')')]
        tokens = [x for x in tokens if not x == ',']       
        tokens = [x for x in tokens if not x == '?']
        tokens = [x for x in tokens if not x == '!']
        tokens = [x for x in tokens if not x == ':']
        tokens = [x for x in tokens if not x == ';']
        tokens = [x for x in tokens if not x == '"']
        tokens = [x for x in tokens if not x == "'"]
        return tokens
    except:
        print("Unexpected error:", sys.exc_info()[0])
        return 'NC'
    
def tokenizeWithZemberek(tweet):
    try:
        tweet = tweet.lower() 
        tokens = tokenizer.tokenize(tweet)
        tokens = [x for x in tokens if not x.startswith('@')]
        tokens = [x for x in tokens if not x.startswith('#')]
        tokens = [x for x in tokens if not x.startswith('http')]
        tokens = [x for x in tokens if not x.startswith('.')]
        tokens = [x for x in tokens if not x.startswith('(')]
        tokens = [x for x in tokens if not x.startswith(')')]
        tokens = [x for x in tokens if not x == ',']       
        tokens = [x for x in tokens if not x == '?']
        tokens = [x for x in tokens if not x == '!']
        tokens = [x for x in tokens if not x == ':']
        tokens = [x for x in tokens if not x == ';']
        tokens = [x for x in tokens if not x == '"']
        tokens = [x for x in tokens if not x == "'"]
        zemberekTokens = []
        for token in tokens:
            if token.strip()>'':
                zemberekToken = zemberek.kelimeCozumle(token)
                if zemberekToken:
                    zemberekTokens.append(zemberekToken[0].kok().icerik())
                else:
                    zemberekTokens.append(token)
            
        return zemberekTokens
    except:
        print("Unexpected error:", sys.exc_info()[0])
        return 'NC'

    
def postprocess(data):
    data['tokens'] = data['SentimentText'].progress_map(tokenize)
    data['zemberekTokens'] = data['SentimentText'].progress_map(tokenizeWithZemberek)
    data = data[data.tokens != 'NC']
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    return data

df = postprocess(df)

progress-bar: 100%|██████████| 3000/3000 [00:02<00:00, 1099.26it/s]


In [21]:
df.head(10)

,Sentiment,SentimentText,tokens,zemberekTokens
0,1,dun Turkcelle tepkilerimizden sonra bugün Turk...,"[dun, turkcelle, tepkilerimizden, sonra, bugün...","[dun, turkcelle, tepki, sonra, bugün, turkcell..."
1,1,girmezmiyim.. Turkcell kartim bile var.. Yarin...,"[girmezmiyim, turkcell, kartim, bile, var, yar...","[girmezmiyim, turkcell, kartim, bile, var, yar..."
2,1,tam tünelden gecerken 3g cekiyordu:D türkcell'...,"[tam, tünelden, gecerken, 3g, cekiyordu, :d, t...","[tam, tünel, gecerken, 3g, cekiyordu, :d, türk..."
3,1,turkcell superonline fiber internet veya ADSL ...,"[turkcell, superonline, fiber, internet, veya,...","[turkcell, superonline, fiber, internet, veya,..."
4,1,bence Gnçtrkcll Ark Winterfest 2012'de 1.olur ...,"[bence, gnçtrkcll, ark, winterfest, 2012, de, ...","[ben, gnçtrkcll, ark, winterfest, 2012, de, ol..."
5,1,turkcell sana kıyak geçiyor :D,"[turkcell, sana, kıyak, geçiyor, :d]","[turkcell, sen, kıyak, geç, :d]"
6,1,Turkcelle bağlan hayata diyorum ;))),"[turkcelle, bağlan, hayata, diyorum, ;)]","[turkcelle, bağla, hayat, de, ;)]"
7,1,"Kimse takmıyo beni yaaaaa, turkcell bana mesaj...","[kimse, takmıyo, beni, yaaaaa, turkcell, bana,...","[kimse, takmıyo, ben, yaaaaa, turkcell, ben, m..."
8,1,Bu Turkcell pusula uygulaması iPhone kullanıcı...,"[bu, turkcell, pusula, uygulaması, iphone, kul...","[bu, turkcell, pusula, uygula, iphone, kullan,..."
9,1,o zaman sorun yok bende turkcell için iyi bir ...,"[o, zaman, sorun, yok, bende, turkcell, için, ...","[o, zaman, sorun, yok, bent, turkcell, için, i..."


# Helmotz Weighting Functions

In [22]:
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

def combination(m,n):
    return factorial(m)/(factorial(n)*factorial(m-n))

def computeMeaning(wordDictX,wordDictC):
    meaningDict = {}
    
    L = 0
    B = 0
    
    for key, value in wordDictX.items(): # key is word , a word is used how many times 
        B = B + value
    
    for key, value in wordDictC.items(): #all corpus
        L = L + value

    for word, count in wordDictX.items():
        if count == 0:
            meaningDict[word] = (count-1)*math.log10(L/B)
        else:
            meaningDict[word] = meaningDict[word] = (-1/count)*(math.log10(combination(wordDictC[word],count)))- ((count-1)*math.log10(L/B))
    return meaningDict    

# Helmotz Weighting Implementation

In [23]:
helmotzDic = {}
helmotzZemberekDic = {}

helmotzDocumentDics = []
helmotzZemberekDocumentDics = []

for index, row in df.iterrows():
    dic = {}
    zemberekDic = {}
    for token in row['tokens']:
        if token in dic:
            dic[token] = dic[token] + 1
        else:
            dic[token] = 1
            
        if token in helmotzDic:
            helmotzDic[token] = helmotzDic[token] + 1
        else:
            helmotzDic[token] = 1
            
    for token in row['zemberekTokens']:
        if token in zemberekDic:
            zemberekDic[token] = zemberekDic[token] + 1
        else:
            zemberekDic[token] = 1
            
        if token in helmotzZemberekDic:
            helmotzZemberekDic[token] = helmotzZemberekDic[token] + 1
        else:
            helmotzZemberekDic[token] = 1
    
    helmotzDocumentDics.append(dic)
    helmotzZemberekDocumentDics.append(zemberekDic)


In [24]:
helmontzMeanings = []
helmontzZemberekMeanings = []

for helmotzDocumentDic in helmotzDocumentDics:
    helmontzMeanings.append(computeMeaning(helmotzDocumentDic, helmotzDic))

for helmotzZemberekDocumentDic in helmotzZemberekDocumentDics:
    helmontzZemberekMeanings.append(computeMeaning(helmotzZemberekDocumentDic, helmotzZemberekDic))

In [25]:
helmontzMeaningDataFrame=  pd.DataFrame(helmontzMeanings)
helmontzZemberekMeaningDataFrame=  pd.DataFrame(helmontzZemberekMeanings)

In [26]:
#the average of the values of each word in documents is calculated.
helmotzDic = helmontzMeaningDataFrame.mean(axis=0).sort_values().to_dict()
helmotzZemberekDic = helmontzZemberekMeaningDataFrame.mean(axis=0).sort_values().to_dict()

# Building word2vec model

In [27]:
x_train, x_test, y_train, y_test = train_test_split(np.array(df.tokens),
                                                    np.array(df.Sentiment), test_size=0.2)

x_train_zemberek, x_test_zemberek, y_train_zemberek, y_test_zemberek = train_test_split(np.array(df.zemberekTokens),
                                                    np.array(df.Sentiment), test_size=0.2)

In [28]:
def labelizeTweets(tweets, label_type):
    labelized = []
    for i,v in tqdm(enumerate(tweets)):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

x_train = labelizeTweets(x_train, 'TRAIN')
x_test = labelizeTweets(x_test, 'TEST')

x_train_zemberek = labelizeTweets(x_train_zemberek, 'TRAIN')
x_test_zemberek = labelizeTweets(x_test_zemberek, 'TEST')

0it [00:00, ?it/s]C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """
2400it [00:00, 122784.07it/s]
600it [00:00, 133011.75it/s]
2400it [00:00, 177314.64it/s]
600it [00:00, 62993.30it/s]


In [30]:
n_dims = [50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300]
n_dim = 200
tweet_w2v = Word2Vec(size=n_dim, min_count=3 ,hs=1, window = 7, iter = 100, sg = 0)
tweet_w2v.build_vocab([x.words for x in tqdm(x_train)])
tweet_w2v.train([x.words for x in tqdm(x_train)],total_examples=tweet_w2v.corpus_count, epochs=tweet_w2v.iter)

tweet_w2v_zemberek = Word2Vec(size=n_dim, min_count=3 ,hs=1, window = 7, iter = 100, sg = 0)
tweet_w2v_zemberek.build_vocab([x.words for x in tqdm(x_train_zemberek)])
tweet_w2v_zemberek.train([x.words for x in tqdm(x_train_zemberek)],total_examples=tweet_w2v_zemberek.corpus_count, epochs=tweet_w2v_zemberek.iter)



tweet_w2v_sg = Word2Vec(size=n_dim, min_count=3 ,hs=1, window = 7, iter = 100, sg =1)
tweet_w2v_sg.build_vocab([x.words for x in tqdm(x_train)])
tweet_w2v_sg.train([x.words for x in tqdm(x_train)],total_examples=tweet_w2v_sg.corpus_count, epochs=tweet_w2v_sg.iter)

tweet_w2v_zemberek_sg = Word2Vec(size=n_dim, min_count=3 ,hs=1, window = 7, iter = 100, sg =1)
tweet_w2v_zemberek_sg.build_vocab([x.words for x in tqdm(x_train_zemberek)])
tweet_w2v_zemberek_sg.train([x.words for x in tqdm(x_train_zemberek)],total_examples=tweet_w2v_zemberek_sg.corpus_count, epochs=tweet_w2v_zemberek_sg.iter)


100%|██████████| 2400/2400 [00:00<00:00, 794250.40it/s]
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """
100%|██████████| 2400/2400 [00:00<00:00, 798851.65it/s]
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  if __name__ == '__main__':
100%|██████████| 2400/2400 [00:00<00:00, 684086.28it/s]
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  from ipykernel import kernelapp as app
100%|██████████| 2400/2400 [00:00<00:00, 1196805.33it/s]
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs ins

(1790200, 2915000)

# Classifier

In [31]:
print('building tf-idf matrix ...')
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=3)
matrix = vectorizer.fit_transform([x.words for x in x_train])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print('vocab size :', len(tfidf))


print('building tf-idf matrix zemberek ...')
vectorizer_zemberek = TfidfVectorizer(analyzer=lambda x: x, min_df=3)
matrix_zemberek = vectorizer_zemberek.fit_transform([x.words for x in x_train_zemberek])
tfidf_zemberek = dict(zip(vectorizer_zemberek.get_feature_names(), vectorizer_zemberek.idf_))
print('vocab size zemberek :', len(tfidf_zemberek))




print('building tf-idf matrix sg...')
vectorizer_sg = TfidfVectorizer(analyzer=lambda x: x, min_df=3)
matrix_sg = vectorizer_sg.fit_transform([x.words for x in x_train])
tfidf_sg = dict(zip(vectorizer_sg.get_feature_names(), vectorizer_sg.idf_))
print('vocab size sg :', len(tfidf_sg))


print('building tf-idf matrix zemberek sg ...')
vectorizer_zemberek_sg = TfidfVectorizer(analyzer=lambda x: x, min_df=3)
matrix_zemberek_sg = vectorizer_zemberek_sg.fit_transform([x.words for x in x_train_zemberek])
tfidf_zemberek_sg = dict(zip(vectorizer_zemberek_sg.get_feature_names(), vectorizer_zemberek_sg.idf_))
print('vocab size zemberek sg :', len(tfidf_zemberek_sg))

building tf-idf matrix ...
vocab size : 1480
building tf-idf matrix zemberek ...
vocab size zemberek : 1369
building tf-idf matrix sg...
vocab size sg : 1480
building tf-idf matrix zemberek sg ...
vocab size zemberek sg : 1369


In [32]:
def buildWordVector(tokens, size, multiplyBy, isZemberek, isSg):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            if multiplyBy == "tfidf":
                if isZemberek:
                    if isSg:
                        vec += tweet_w2v_zemberek_sg[word].reshape((1, size)) * tfidf_zemberek_sg[word]
                    else:
                        vec += tweet_w2v_zemberek[word].reshape((1, size)) * tfidf_zemberek[word]
                else:
                    if isSg:
                        vec += tweet_w2v_sg[word].reshape((1, size)) * tfidf_sg[word]
                    else:
                        vec += tweet_w2v[word].reshape((1, size)) * tfidf[word]
            elif multiplyBy == "helmotz":
                if isZemberek:
                    if isSg:
                        vec += tweet_w2v_zemberek_sg[word].reshape((1, size)) * helmotzZemberekDic[word]
                    else:
                        vec += tweet_w2v_zemberek[word].reshape((1, size)) * helmotzZemberekDic[word]
                else:
                    if isSg:
                        vec += tweet_w2v_sg[word].reshape((1, size)) * helmotzDic[word]
                    else:
                        vec += tweet_w2v[word].reshape((1, size)) * helmotzDic[word]
            else:
                if isZemberek:
                    if isSg:
                        vec += tweet_w2v_zemberek_sg[word].reshape((1, size))
                    else:
                        vec += tweet_w2v_zemberek[word].reshape((1, size))  
                else:
                    if isSg:
                        vec += tweet_w2v_sg[word].reshape((1, size))
                    else:
                        vec += tweet_w2v[word].reshape((1, size))
       
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [33]:
#without weighting
train_vecs_w2v = np.concatenate([buildWordVector(z, n_dim, "none", False, False) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, n_dim, "none", False, False) for z in tqdm(map(lambda x: x.words, x_test))])
test_vecs_w2v = scale(test_vecs_w2v)


#without weighting, with zemberek
train_vecs_w2v_zemberek = np.concatenate([buildWordVector(z, n_dim, "none", True, False) for z in tqdm(map(lambda x: x.words, x_train_zemberek))])
train_vecs_w2v_zemberek = scale(train_vecs_w2v_zemberek)

test_vecs_w2v_zemberek = np.concatenate([buildWordVector(z, n_dim, "none", True, False) for z in tqdm(map(lambda x: x.words, x_test_zemberek))])
test_vecs_w2v_zemberek = scale(test_vecs_w2v_zemberek)



#with tfidf
train_vecs_w2v_tfidf = np.concatenate([buildWordVector(z, n_dim, "tfidf", False, False) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_w2v_tfidf = scale(train_vecs_w2v_tfidf)

test_vecs_w2v_tfidf = np.concatenate([buildWordVector(z, n_dim, "tfidf", False, False) for z in tqdm(map(lambda x: x.words, x_test))])
test_vecs_w2v_tfidf = scale(test_vecs_w2v_tfidf)


#with tfidf and zemberek
train_vecs_w2v_zemberek_tfidf = np.concatenate([buildWordVector(z, n_dim, "tfidf", True, False) for z in tqdm(map(lambda x: x.words, x_train_zemberek))])
train_vecs_w2v_zemberek_tfidf = scale(train_vecs_w2v_zemberek_tfidf)

test_vecs_w2v_zemberek_tfidf = np.concatenate([buildWordVector(z, n_dim, "tfidf", True, False) for z in tqdm(map(lambda x: x.words, x_test_zemberek))])
test_vecs_w2v_zemberek_tfidf = scale(test_vecs_w2v_zemberek_tfidf)



#with helmotz
train_vecs_w2v_helmotz = np.concatenate([buildWordVector(z, n_dim, "helmotz", False, False) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_w2v_helmotz = scale(train_vecs_w2v_helmotz)

test_vecs_w2v_helmotz = np.concatenate([buildWordVector(z, n_dim, "helmotz", False, False) for z in tqdm(map(lambda x: x.words, x_test))])
test_vecs_w2v_helmotz = scale(test_vecs_w2v_helmotz)


#with helmotz and zemberek
train_vecs_w2v_zemberek_helmotz = np.concatenate([buildWordVector(z, n_dim, "helmotz", True, False) for z in tqdm(map(lambda x: x.words, x_train_zemberek))])
train_vecs_w2v_zemberek_helmotz = scale(train_vecs_w2v_zemberek_tfidf)

test_vecs_w2v_zemberek_helmotz = np.concatenate([buildWordVector(z, n_dim, "helmotz", True, False) for z in tqdm(map(lambda x: x.words, x_test_zemberek))])
test_vecs_w2v_zemberek_helmotz = scale(test_vecs_w2v_zemberek_helmotz)



#without weighting but with sg
train_vecs_w2v_sg = np.concatenate([buildWordVector(z, n_dim, "none", False, True) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_w2v_sg = scale(train_vecs_w2v_sg)

test_vecs_w2v_sg = np.concatenate([buildWordVector(z, n_dim, "none", False, True) for z in tqdm(map(lambda x: x.words, x_test))])
test_vecs_w2v_sg = scale(test_vecs_w2v_sg)


#without weighting but with sg and zemberek
train_vecs_w2v_zemberek_sg = np.concatenate([buildWordVector(z, n_dim, "none", True, True) for z in tqdm(map(lambda x: x.words, x_train_zemberek))])
train_vecs_w2v_zemberek_sg = scale(train_vecs_w2v_zemberek_sg)

test_vecs_w2v_zemberek_sg = np.concatenate([buildWordVector(z, n_dim, "none", True, True) for z in tqdm(map(lambda x: x.words, x_test_zemberek))])
test_vecs_w2v_zemberek_sg = scale(test_vecs_w2v_zemberek_sg)



#with tfidf and sg
train_vecs_w2v_tfidf_sg = np.concatenate([buildWordVector(z, n_dim, "tfidf", False, True) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_w2v_tfidf_sg = scale(train_vecs_w2v_tfidf_sg)

test_vecs_w2v_tfidf_sg = np.concatenate([buildWordVector(z, n_dim, "tfidf", False, True) for z in tqdm(map(lambda x: x.words, x_test))])
test_vecs_w2v_tfidf_sg = scale(test_vecs_w2v_tfidf_sg)


#with tfidf,zemberek and sg
train_vecs_w2v_zemberek_tfidf_sg = np.concatenate([buildWordVector(z, n_dim, "tfidf", True, True) for z in tqdm(map(lambda x: x.words, x_train_zemberek))])
train_vecs_w2v_zemberek_tfidf_sg = scale(train_vecs_w2v_zemberek_tfidf_sg)

test_vecs_w2v_zemberek_tfidf_sg = np.concatenate([buildWordVector(z, n_dim, "tfidf", True, True) for z in tqdm(map(lambda x: x.words, x_test_zemberek))])
test_vecs_w2v_zemberek_tfidf_sg = scale(test_vecs_w2v_zemberek_tfidf_sg)


#with helmotz and sg
train_vecs_w2v_helmotz_sg = np.concatenate([buildWordVector(z, n_dim, "helmotz", False, True) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_w2v_helmotz_sg = scale(train_vecs_w2v_helmotz_sg)

test_vecs_w2v_helmotz_sg = np.concatenate([buildWordVector(z, n_dim, "helmotz", False, True) for z in tqdm(map(lambda x: x.words, x_test))])
test_vecs_w2v_helmotz_sg = scale(test_vecs_w2v_helmotz_sg)


#with helmotz, zemberek and sg
train_vecs_w2v_zemberek_helmotz_sg = np.concatenate([buildWordVector(z, n_dim, "helmotz", True, True) for z in tqdm(map(lambda x: x.words, x_train_zemberek))])
train_vecs_w2v_zemberek_helmotz_sg = scale(train_vecs_w2v_zemberek_helmotz_sg)

test_vecs_w2v_zemberek_helmotz_sg = np.concatenate([buildWordVector(z, n_dim, "helmotz", True, True) for z in tqdm(map(lambda x: x.words, x_test_zemberek))])
test_vecs_w2v_zemberek_helmotz_sg = scale(test_vecs_w2v_zemberek_helmotz_sg)



0it [00:00, ?it/s]C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
2400it [00:00, 8752.25it/s]
600it [00:00, 6999.53it/s]
0it [00:00, ?it/s]C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
2400it [00:00, 6067.39it/s]
600it [00:00, 6918.40it/s]
0it [00:00, ?it/s]C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()
2400it [00:00, 6469.57it/s]
600it [00:00, 6201.53it/s]
0it [00:00, ?it/s]C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use sel

In [34]:
classifiers = ["SVM", "NaiveBayes", "DecisionTree", "MLP", "KNN", "Voting"]
tfidfs = [0, 1, 2]  #without weighting(0), tfidf(1), helmotz(2)
zembereks = [0, 1]
sgs = [0,1]

for sg in sgs:
    for tf in tfidfs:
        for zemberek in zembereks:
            for classifier in classifiers:
                cla = None
                if classifier == "SVM":
                    cla = svm.SVC()
                if classifier == "NaiveBayes":
                    cla = GaussianNB()
                if classifier == "DecisionTree":
                    cla = DecisionTreeClassifier()
                if classifier == "MLP":
                    cla = MLPClassifier()
                if classifier == "KNN":
                    cla = KNeighborsClassifier()
                if classifier == "Voting":
                    cla = VotingClassifier(estimators=[('nb', GaussianNB()), ('svm', svm.LinearSVC()), ('mlp', MLPClassifier()),('knn', KNeighborsClassifier())], voting='hard')
                if sg == 0:
                    if tf == 0 and zemberek ==0:
                        cla.fit(train_vecs_w2v, y_train.astype('int'))
                        y_pred = cla.predict(test_vecs_w2v)
                        score = accuracy_score(y_test.astype('int'), y_pred)
                    elif tf == 0 and zemberek ==1:
                        cla.fit(train_vecs_w2v_zemberek, y_train_zemberek.astype('int'))
                        y_pred = cla.predict(test_vecs_w2v_zemberek)
                        score = accuracy_score(y_test_zemberek.astype('int'), y_pred)
                    elif tf == 1 and zemberek ==0:
                        cla.fit(train_vecs_w2v_tfidf, y_train.astype('int'))
                        y_pred = cla.predict(test_vecs_w2v_tfidf)
                        score = accuracy_score(y_test.astype('int'), y_pred)
                    elif tf == 1 and zemberek ==1:
                        cla.fit(train_vecs_w2v_zemberek_tfidf, y_train_zemberek.astype('int'))
                        y_pred = cla.predict(test_vecs_w2v_zemberek_tfidf)
                        score = accuracy_score(y_test_zemberek.astype('int'), y_pred)
                    elif tf == 2 and zemberek ==0:
                        cla.fit(train_vecs_w2v_helmotz, y_train.astype('int'))
                        y_pred = cla.predict(test_vecs_w2v_helmotz)
                        score = accuracy_score(y_test.astype('int'), y_pred)
                    elif tf == 2 and zemberek ==1:
                        cla.fit(train_vecs_w2v_zemberek_helmotz, y_train_zemberek.astype('int'))
                        y_pred = cla.predict(test_vecs_w2v_zemberek_helmotz)
                        score = accuracy_score(y_test_zemberek.astype('int'), y_pred)
                if sg == 1:
                    if tf == 0 and zemberek ==0:
                        cla.fit(train_vecs_w2v_sg, y_train.astype('int'))
                        y_pred = cla.predict(test_vecs_w2v_sg)
                        score = accuracy_score(y_test.astype('int'), y_pred)
                    elif tf == 0 and zemberek ==1:
                        cla.fit(train_vecs_w2v_zemberek_sg, y_train_zemberek.astype('int'))
                        y_pred = cla.predict(test_vecs_w2v_zemberek_sg)
                        score = accuracy_score(y_test_zemberek.astype('int'), y_pred)
                    elif tf == 1 and zemberek ==0:
                        cla.fit(train_vecs_w2v_tfidf_sg, y_train.astype('int'))
                        y_pred = cla.predict(test_vecs_w2v_tfidf_sg)
                        score = accuracy_score(y_test.astype('int'), y_pred)
                    elif tf == 1 and zemberek ==1:
                        cla.fit(train_vecs_w2v_zemberek_tfidf_sg, y_train_zemberek.astype('int'))
                        y_pred = cla.predict(test_vecs_w2v_zemberek_tfidf_sg)
                        score = accuracy_score(y_test_zemberek.astype('int'), y_pred)
                    elif tf == 2 and zemberek ==0:
                        cla.fit(train_vecs_w2v_helmotz_sg, y_train.astype('int'))
                        y_pred = cla.predict(test_vecs_w2v_helmotz_sg)
                        score = accuracy_score(y_test.astype('int'), y_pred)
                    elif tf == 2 and zemberek ==1:
                        cla.fit(train_vecs_w2v_zemberek_helmotz_sg, y_train_zemberek.astype('int'))
                        y_pred = cla.predict(test_vecs_w2v_zemberek_helmotz_sg)
                        score = accuracy_score(y_test_zemberek.astype('int'), y_pred)
                print(classifier, " (tfidf=", tf, ") ( sg =", sg, " ) ( zemberek=", zemberek  , "):", score)


SVM  (tfidf= 0 ) ( sg = 0  ) ( zemberek= 0 ): 0.54
NaiveBayes  (tfidf= 0 ) ( sg = 0  ) ( zemberek= 0 ): 0.5066666666666667
DecisionTree  (tfidf= 0 ) ( sg = 0  ) ( zemberek= 0 ): 0.42
MLP  (tfidf= 0 ) ( sg = 0  ) ( zemberek= 0 ): 0.49666666666666665
KNN  (tfidf= 0 ) ( sg = 0  ) ( zemberek= 0 ): 0.455


C:\Users\asus\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Voting  (tfidf= 0 ) ( sg = 0  ) ( zemberek= 0 ): 0.5383333333333333
SVM  (tfidf= 0 ) ( sg = 0  ) ( zemberek= 1 ): 0.5383333333333333
NaiveBayes  (tfidf= 0 ) ( sg = 0  ) ( zemberek= 1 ): 0.5116666666666667
DecisionTree  (tfidf= 0 ) ( sg = 0  ) ( zemberek= 1 ): 0.37833333333333335
MLP  (tfidf= 0 ) ( sg = 0  ) ( zemberek= 1 ): 0.5133333333333333
KNN  (tfidf= 0 ) ( sg = 0  ) ( zemberek= 1 ): 0.44333333333333336


C:\Users\asus\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Voting  (tfidf= 0 ) ( sg = 0  ) ( zemberek= 1 ): 0.5466666666666666
SVM  (tfidf= 1 ) ( sg = 0  ) ( zemberek= 0 ): 0.5333333333333333
NaiveBayes  (tfidf= 1 ) ( sg = 0  ) ( zemberek= 0 ): 0.495
DecisionTree  (tfidf= 1 ) ( sg = 0  ) ( zemberek= 0 ): 0.41833333333333333
MLP  (tfidf= 1 ) ( sg = 0  ) ( zemberek= 0 ): 0.495
KNN  (tfidf= 1 ) ( sg = 0  ) ( zemberek= 0 ): 0.44


...

Voting  (tfidf= 1 ) ( sg = 1  ) ( zemberek= 1 ): 0.5616666666666666
SVM  (tfidf= 2 ) ( sg = 1  ) ( zemberek= 0 ): 0.555
NaiveBayes  (tfidf= 2 ) ( sg = 1  ) ( zemberek= 0 ): 0.4816666666666667
DecisionTree  (tfidf= 2 ) ( sg = 1  ) ( zemberek= 0 ): 0.4116666666666667
MLP  (tfidf= 2 ) ( sg = 1  ) ( zemberek= 0 ): 0.5166666666666667
KNN  (tfidf= 2 ) ( sg = 1  ) ( zemberek= 0 ): 0.49666666666666665


C:\Users\asus\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Voting  (tfidf= 2 ) ( sg = 1  ) ( zemberek= 0 ): 0.5733333333333334
SVM  (tfidf= 2 ) ( sg = 1  ) ( zemberek= 1 ): 0.56
NaiveBayes  (tfidf= 2 ) ( sg = 1  ) ( zemberek= 1 ): 0.48333333333333334
DecisionTree  (tfidf= 2 ) ( sg = 1  ) ( zemberek= 1 ): 0.42
MLP  (tfidf= 2 ) ( sg = 1  ) ( zemberek= 1 ): 0.5416666666666666
KNN  (tfidf= 2 ) ( sg = 1  ) ( zemberek= 1 ): 0.4866666666666667


C:\Users\asus\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Voting  (tfidf= 2 ) ( sg = 1  ) ( zemberek= 1 ): 0.545
